In [1]:
# import pandas as pd
# import altair as alt
# alt.data_transformers.disable_max_rows()
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# !pip install pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
from csv import reader
from pyspark.sql.types import DoubleType,StringType,IntegerType,DateType
from pyspark.sql.functions import col,when,count,desc
from pyspark.sql.functions import format_string
import sys
from pyspark.sql import functions as F
from pyspark.sql.types import *
import pandas as pd

In [3]:
sc=SparkContext.getOrCreate()
spark = SparkSession(sc)
# e/tables/Men_T20I_Player_Innings_Stats___21st_Century-fd5cb.csv
data_df = spark.read.format("csv").option("header", "true").load('/FileStore/tables//Men_T20I_Player_Innings_Stats___21st_Century-fd5cb.csv')



In [4]:
data_df.columns


Out[4]: ['Innings Player',
 'Innings Runs Scored',
 'Innings Runs Scored Num',
 'Innings Minutes Batted',
 'Innings Batted Flag',
 'Innings Not Out Flag',
 'Innings Balls Faced',
 'Innings Boundary Fours',
 'Innings Boundary Sixes',
 'Innings Batting Strike Rate',
 'Innings Number',
 'Opposition',
 'Ground',
 'Innings Date',
 'Country',
 "50's",
 "100's",
 'Innings Runs Scored Buckets',
 'Innings Overs Bowled',
 'Innings Bowled Flag',
 'Innings Maidens Bowled',
 'Innings Runs Conceded',
 'Innings Wickets Taken',
 '4 Wickets',
 '5 Wickets',
 '10 Wickets',
 'Innings Wickets Taken Buckets',
 'Innings Economy Rate']

In [5]:
averagedf=data_df.select(['Innings Player','Innings Batting Strike Rate','Innings Economy Rate'])
# averagedf=averagedf.orderby('Innings Player')

In [6]:

cols=['Innings Batting Strike Rate','Innings Economy Rate']
for col_name in cols:
    averagedf = averagedf.withColumn(col_name, col(col_name).cast('float'))
averagedf.dtypes

Out[6]: [('Innings Player', 'string'),
 ('Innings Batting Strike Rate', 'float'),
 ('Innings Economy Rate', 'float')]

In [7]:

results =averagedf.groupBy('Innings Player').mean() 
results=results.withColumnRenamed("Innings Player", "Player_Name")
# results.show()
results.count()

Out[7]: 1080

In [8]:
del_df = spark.read.format("csv").option("header", "true").load('/FileStore/tables//Player.csv')
del_df.show()

+---------+---------------+----------+------------+--------------------+------------+
Player_Id| Player_Name| DOB|Batting_Hand| Bowling_Skill| Country|
+---------+---------------+----------+------------+--------------------+------------+
 1| SC Ganguly| 7/8/1972| Left_Hand| Right-arm medium| India|
 2| BB McCullum| 9/27/1981| Right_Hand| Right-arm medium| New Zealand|
 3| RT Ponting|12/19/1974| Right_Hand| Right-arm medium| Australia|
 4| DJ Hussey| 7/15/1977| Right_Hand| Right-arm offbreak| Australia|
 5|Mohammad Hafeez|10/17/1980| Right_Hand| Right-arm offbreak| Pakistan|
 6| R Dravid| 1/11/1973| Right_Hand| Right-arm offbreak| India|
 7| W Jaffer| 2/16/1978| Right_Hand| Right-arm offbreak| India|
 8| V Kohli| 11/5/1988| Right_Hand| Right-arm medium| India|
 9| JH Kallis|10/16/1975| Right_Hand|Right-arm fast-me...|South Africa|
 10| CL White| 8/18/1983| Right_Hand| Legbreak googly| Australia|
 11| MV Boucher| 12/3/1976| Right_Hand| Right-arm medium|South Africa|
 12| B Akhil| 10/7/1977| Right_Hand|Right-arm medium-...| India|
 13| AA Noffke| 4/30/1977| Right_Hand|Right-arm fast-me...| Australia|
 14| P Kumar| 10/2/1986| Right_Hand| Right-arm medium| India|
 15| Z Khan| 10/7/1978| Right_Hand|Left-arm fast-medium| India|
 16| SB Joshi| 6/6/1970| Left_Hand|Slow left-arm ort...| India|
 17| PA Patel| 3/9/1985| Left_Hand| NULL| India|
 18| ML Hayden|10/29/1971| Left_Hand| Right-arm medium| Australia|
 19| MEK Hussey| 5/27/1975| Left_Hand| Right-arm medium| Australia|
 20| MS Dhoni| 7/7/1981| Right_Hand| Right-arm medium| India|
+---------+---------------+----------+------------+--------------------+------------+
only showing top 20 rows

In [9]:
split_col = F.split(del_df['DOB'], '/')
del_df = del_df.withColumn('DOB', split_col.getItem(2))


del_df = del_df.withColumn("DOB", col("DOB").cast('float'))


del_df=del_df.withColumn("age",2020-del_df["DOB"])
del_df.show()
# del_df=del_df.withColumn("age", F.datediff("DOB", "current_date")/365)
# full_outer_join = del_df.join(results, results["Innings Player"] == del_df["Player_Name"],how='full') # Could also use 'full_outer'

# results=results.toPandas()
# joined_df = results.join(del_df, on=['Player_Name'], how='outer')
# joined_df.toPandas()
# full_outer_join.show()

+---------+---------------+------+------------+--------------------+------------+----+
Player_Id| Player_Name| DOB|Batting_Hand| Bowling_Skill| Country| age|
+---------+---------------+------+------------+--------------------+------------+----+
 1| SC Ganguly|1972.0| Left_Hand| Right-arm medium| India|48.0|
 2| BB McCullum|1981.0| Right_Hand| Right-arm medium| New Zealand|39.0|
 3| RT Ponting|1974.0| Right_Hand| Right-arm medium| Australia|46.0|
 4| DJ Hussey|1977.0| Right_Hand| Right-arm offbreak| Australia|43.0|
 5|Mohammad Hafeez|1980.0| Right_Hand| Right-arm offbreak| Pakistan|40.0|
 6| R Dravid|1973.0| Right_Hand| Right-arm offbreak| India|47.0|
 7| W Jaffer|1978.0| Right_Hand| Right-arm offbreak| India|42.0|
 8| V Kohli|1988.0| Right_Hand| Right-arm medium| India|32.0|
 9| JH Kallis|1975.0| Right_Hand|Right-arm fast-me...|South Africa|45.0|
 10| CL White|1983.0| Right_Hand| Legbreak googly| Australia|37.0|
 11| MV Boucher|1976.0| Right_Hand| Right-arm medium|South Africa|44.0|
 12| B Akhil|1977.0| Right_Hand|Right-arm medium-...| India|43.0|
 13| AA Noffke|1977.0| Right_Hand|Right-arm fast-me...| Australia|43.0|
 14| P Kumar|1986.0| Right_Hand| Right-arm medium| India|34.0|
 15| Z Khan|1978.0| Right_Hand|Left-arm fast-medium| India|42.0|
 16| SB Joshi|1970.0| Left_Hand|Slow left-arm ort...| India|50.0|
 17| PA Patel|1985.0| Left_Hand| NULL| India|35.0|
 18| ML Hayden|1971.0| Left_Hand| Right-arm medium| Australia|49.0|
 19| MEK Hussey|1975.0| Left_Hand| Right-arm medium| Australia|45.0|
 20| MS Dhoni|1981.0| Right_Hand| Right-arm medium| India|39.0|
+---------+---------------+------+------------+--------------------+------------+----+
only showing top 20 rows

In [10]:
list1.join(list2, "id", "left").show()

In [11]:
display(joined_df)

In [12]:
joined_df.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/joined_df.csv")

In [13]:
del_df.show()

+---------+---------------+----+------------+--------------------+------------+------------+----+
Player_Id| Player_Name| DOB|Batting_Hand| Bowling_Skill| Country|current_date| age|
+---------+---------------+----+------------+--------------------+------------+------------+----+
 1| SC Ganguly|null| Left_Hand| Right-arm medium| India| 2020-05-06|null|
 2| BB McCullum|null| Right_Hand| Right-arm medium| New Zealand| 2020-05-06|null|
 3| RT Ponting|null| Right_Hand| Right-arm medium| Australia| 2020-05-06|null|
 4| DJ Hussey|null| Right_Hand| Right-arm offbreak| Australia| 2020-05-06|null|
 5|Mohammad Hafeez|null| Right_Hand| Right-arm offbreak| Pakistan| 2020-05-06|null|
 6| R Dravid|null| Right_Hand| Right-arm offbreak| India| 2020-05-06|null|
 7| W Jaffer|null| Right_Hand| Right-arm offbreak| India| 2020-05-06|null|
 8| V Kohli|null| Right_Hand| Right-arm medium| India| 2020-05-06|null|
 9| JH Kallis|null| Right_Hand|Right-arm fast-me...|South Africa| 2020-05-06|null|
 10| CL White|null| Right_Hand| Legbreak googly| Australia| 2020-05-06|null|
 11| MV Boucher|null| Right_Hand| Right-arm medium|South Africa| 2020-05-06|null|
 12| B Akhil|null| Right_Hand|Right-arm medium-...| India| 2020-05-06|null|
 13| AA Noffke|null| Right_Hand|Right-arm fast-me...| Australia| 2020-05-06|null|
 14| P Kumar|null| Right_Hand| Right-arm medium| India| 2020-05-06|null|
 15| Z Khan|null| Right_Hand|Left-arm fast-medium| India| 2020-05-06|null|
 16| SB Joshi|null| Left_Hand|Slow left-arm ort...| India| 2020-05-06|null|
 17| PA Patel|null| Left_Hand| NULL| India| 2020-05-06|null|
 18| ML Hayden|null| Left_Hand| Right-arm medium| Australia| 2020-05-06|null|
 19| MEK Hussey|null| Left_Hand| Right-arm medium| Australia| 2020-05-06|null|
 20| MS Dhoni|null| Right_Hand| Right-arm medium| India| 2020-05-06|null|
+---------+---------------+----+------------+--------------------+------------+------------+----+
only showing top 20 rows